# Merge Noodle

In [1]:
import pandas as pd
from merge import on_intervals, AST
from util.signed_overlap import overlap

In [2]:
left = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "left_measure","left_category"],
    data=[
        ["H001", "L",   0, 100, 55, "b"],
        ["H001", "L", 100, 200, 57, "a"],
        ["H001", "L", 200, 300, 25, "b"],
        ["H001", "L", 300, 400, 94, "e"],
    ]
)

right = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "right_measure", "right_category"],
    data=[
        ["H001", "L", 50, 140, 1.0, "A"],  # 50  40   0  0
        ["H001", "L", 140, 160, 2.0, "B"],  # 0  20   0  0
        ["H001", "L", 160, 180, 3.0, "B"],  # 0  20   0  0
        ["H001", "L", 180, 220, 4.0, "B"],  # 0  20  20  0
        ["H001", "L", 220, 240, 5.0, "C"],  # 0   0  20  0
        ["H001", "L", 240, 260, 5.0, "C"],  # 0   0  20  0
        ["H001", "L", 260, 280, 6.0, "D"],  # 0   0  20  0
        ["H001", "L", 280, 300, 7.0, "E"],  # 0   0  20  0
        ["H001", "L", 300, 320, 8.0, "F"],  # 0   0     20
    ]
)

expected_output = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "measure longest segment", "measure longest value", "category longest segment", "category longest value"],
    data=[
        ["H001", "L", 0, 100, 1.0, 1.0, "A", "A"],
        ["H001", "L", 100, 200, 1.0, 1.0, "A", "B"],
        ["H001", "L", 200, 300, 4.0, 5.0, "B", "C"],
        ["H001", "L", 300, 400, 8.0, 8.0, "F", "F"],
    ]
)

In [3]:
signed_overlap = pd.DataFrame(overlap(
    left["slk_from"].to_numpy(),
    left["slk_to"].to_numpy(),
    right["slk_from"].to_numpy(),
    right["slk_to"].to_numpy()
).transpose())
signed_overlap

,0,1,2,3,4,5,6,7,8
0,50,-40,-60,-80,-120,-140,-160,-180,-200
1,40,20,20,20,-20,-40,-60,-80,-100
2,-60,-40,-20,20,20,20,20,20,0
3,-160,-140,-120,-80,-60,-40,-20,0,20


In [4]:
AST.compare_equal(AST.left_column("30")*5,AST.left_column("30")*5)

True

In [5]:
tree = (AST.left_column("A")+5)*2 + (AST.left_column("A")+5)*2*3 - (AST.left_column("A")+5)*2
print(tree.to_string())


[-]
 ┠╴[+]
 ┃  ┠╴[*]
 ┃  ┃  ┠╴[+]
 ┃  ┃  ┃  ┠╴[left_column]
 ┃  ┃  ┃  ┃  ┖╴"A"
 ┃  ┃  ┃  ┖╴5
 ┃  ┃  ┖╴2
 ┃  ┖╴[*]
 ┃     ┠╴[*]
 ┃     ┃  ┠╴[+]
 ┃     ┃  ┃  ┠╴[left_column]
 ┃     ┃  ┃  ┃  ┖╴"A"
 ┃     ┃  ┃  ┖╴5
 ┃     ┃  ┖╴2
 ┃     ┖╴3
 ┖╴[*]
    ┠╴[+]
    ┃  ┠╴[left_column]
    ┃  ┃  ┖╴"A"
    ┃  ┖╴5
    ┖╴2


In [6]:
print(AST.execution_plan(tree).to_string())


[execute]
 ┠╴[declare]
 ┃  ┠╴"subtree_2"
 ┃  ┖╴[left_column]
 ┃     ┖╴"A"
 ┠╴[declare]
 ┃  ┠╴"subtree_1"
 ┃  ┖╴[+]
 ┃     ┠╴[refer]
 ┃     ┃  ┖╴"subtree_2"
 ┃     ┖╴5
 ┠╴[declare]
 ┃  ┠╴"subtree_0"
 ┃  ┖╴[*]
 ┃     ┠╴[refer]
 ┃     ┃  ┖╴"subtree_1"
 ┃     ┖╴2
 ┖╴[-]
    ┠╴[+]
    ┃  ┠╴[refer]
    ┃  ┃  ┖╴"subtree_0"
    ┃  ┖╴[*]
    ┃     ┠╴[refer]
    ┃     ┃  ┖╴"subtree_0"
    ┃     ┖╴3
    ┖╴[refer]
       ┖╴"subtree_0"


In [15]:
print(AST.execution_plan(AST.right_column("right_measure").filter((AST.length_of_overlap() > 0)|(AST.length_of_overlap() > 0))).to_string())


[execute]
 ┠╴[declare]
 ┃  ┠╴"subtree_1"
 ┃  ┖╴[length_of_overlap]
 ┠╴[declare]
 ┃  ┠╴"subtree_0"
 ┃  ┖╴[>]
 ┃     ┠╴[refer]
 ┃     ┃  ┖╴"subtree_1"
 ┃     ┖╴0
 ┖╴[filter]
    ┠╴[right_column]
    ┃  ┖╴"right_measure"
    ┖╴[or]
       ┠╴[refer]
       ┃  ┖╴"subtree_0"
       ┖╴[refer]
          ┖╴"subtree_0"


In [8]:
AST.left_column("road") == AST.right_column("road") + AST.left_column("road")

<AST ==>

In [9]:
myast  = AST.right_column("right_category").at_index(
            AST.right_column("right_measure")
            .filter(
                  (AST.right_column("right_measure")>4)
                & (AST.right_column("right_measure")<8)
            ).index_of_max()
)
myast

<AST at_index>

In [10]:
left_columns_required, right_columns_required = AST.columns_required(myast)
left_columns_required, right_columns_required

(set(), {'right_category', 'right_measure'})

In [11]:
AST.output_column_name2(myast)

'right_measure'

In [12]:
[1,2,3][:-1]

[1, 2]

In [13]:
row_of_target:int
row_of_target = 1
AST.evaluate(
    myast,
    left_columns      = left .loc[row_of_target, list(left_columns_required)],
    right_columns     = right.loc[            :, list(right_columns_required)],
    length_of_left    = right.loc[row_of_target, "slk_to"] - right.loc[row_of_target, "slk_from"],
    length_of_right   = left .loc[            :, "slk_to"] - left .loc[            :, "slk_from"],
    length_of_overlap = signed_overlap.iloc[row_of_target,:],
)

series 0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    5.0
6    6.0
7    7.0
8    8.0
Name: right_measure, dtype: float64
mask 0    False
1    False
2    False
3    False
4     True
5     True
6     True
7     True
8    False
Name: right_measure, dtype: bool


'E'

In [14]:
merge(
    target       = segments,
    new_data     = data,
    join_left_on = ["road","cwy"],
    from_to      = ("slk_from","slk_to"),
    add_columns  = [
        column("category").take_index(column("measure").index_of_maximum()).alias("category_of_max_measure"),
        column("category").group()
        column("measure").group_by
    ]
)


SyntaxError: invalid syntax (4048077929.py, line 9)